In [2]:
import numpy as np

import Go_Board
import time
import random
import MCTS_GO

from sgfmill import sgf

import pandas as pd

import tensorflow as tf

from tensorflow.keras import datasets, layers, models

In [3]:
X = []
Y = []

In [4]:
dataset_dir = './go9-large/'#dataset/sgf/train/0000/'

from os import listdir
from os.path import isfile, join
examples = [f for f in listdir(dataset_dir) if isfile(join(dataset_dir, f))]

#example = '2014-12-12T14:25:20.109Z_ls125rvjkot8.sgf'
board_size = 9
examples = examples[:100]

In [5]:
for example in examples:
    board_go = Go_Board.Our_Board(board_size=board_size)
    with open(dataset_dir+example, "rb") as f:
        game = sgf.Sgf_game.from_bytes(f.read())
    winner = game.get_winner()
    board_size = game.get_size()
    root_node = game.get_root()

    board = np.zeros(shape=(board_size,board_size))

    for node in game.get_main_sequence():

        if node.get_move()[0] is not None:
            move = node.get_move()
            player = move[0]
            coord = (move[1][1],move[1][0]) if move[1] is not None else (-1,-1)

            board  = board_go.get_matrix()

            y = np.zeros(shape=(board_size,board_size))
            if(coord[0] != -1 and coord[1] != -1):
                y[coord[0]][coord[1]] = 1
                y = y.reshape(board_size*board_size)
                y = np.append(y,[0])
            else:
                y = np.append(y,[1])

            x = board * -1 if player == 'w' else board
            board_go.push(coord)
            X.append(x)
            Y.append(y)

In [6]:
X = np.reshape(X,(len(X),board_size,board_size,1))

In [7]:
np.shape(X)

(4616, 9, 9, 1)

In [8]:
Y = tf.keras.utils.to_categorical(Y,num_classes=82)

In [9]:
np.shape(Y)

(4616, 82, 82)

In [10]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(9, 9, 1)))
model.add(layers.MaxPooling2D((2, 2)))
"""
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
"""
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(82, activation='softmax'))


In [ ]:
model(X[0])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X, Y, epochs=10)
